In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [51]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [52]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
import random

In [53]:
def load_dataset():
  train_data = pd.read_csv('trainset.csv')
  y_train = pd.read_csv('theta.csv')
  target_data = pd.read_csv('target_data.csv')
  # target_label = pd.read_csv('target_label.csv')
  return train_data, y_train, target_data

In [6]:
# train_data.head()
# y_train.head()

In [54]:
train_data, y_train, target_data = load_dataset()

In [ ]:
# train_data.shape
# target_data.shape

In [55]:
num_classes = 1
train_data = np.array(train_data.iloc[:,:])
train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
target_data = np.array(target_data.iloc[:,:])
target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [56]:
train_data.shape
target_data.shape

(333, 128, 128)

In [57]:
train_data = train_data.astype('float32')
train_data = train_data/255

rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
rgb_batch.shape

target_data = target_data.astype('float32')
target_data = target_data/255

rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
rgb_target.shape

(333, 128, 128, 3)

In [58]:
target_data = rgb_target

In [59]:
target_data.shape

(333, 128, 128, 3)

In [61]:
minx = np.amin(y_train)
print(minx)
maxx = np.amax(y_train)
print(maxx)

theta   -25
dtype: int64
theta    55
dtype: int64


In [62]:
y_norm = (2*(y_train-minx)/(maxx-minx)) - 1

In [63]:
print(y_norm)

     theta
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
..     ...
328   -1.0
329   -1.0
330   -1.0
331   -1.0
332   -1.0

[333 rows x 1 columns]


In [64]:
y_train = y_norm

In [65]:
x_train, x_test, y_train, y_test = train_test_split(rgb_batch, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [66]:
input_shape = (128, 128, 3)

In [ ]:
# print(y_test)

     theta  R1  R2   B
3       55   7   0   2
184     15   7 -25  -3
302    -25  -8   0  12
239     -5  -3   0  -3
50      45  -3   0   2
..     ...  ..  ..  ..
14      55  -8   0   2
253     -5   7 -20   2
193      5 -18   0  12
54      45  -3   0  -3
177     15   7 -10   2

[100 rows x 4 columns]


In [67]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(64, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(128, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    # model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [68]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [69]:
conv = concatenate([current_model, target_model])

In [70]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=1.0)(dense)
dense = Dropout(0.25)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='tanh')(dense)

model = Model(inputs=[current_input, target_input], outputs=[output])

opt = SGD(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])


In [ ]:
best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [71]:
model.fit([x_train, target_xtrain], y_train,
          epochs=30,
          verbose=1)

Epoch 1/30
8/8 [==============================] - 116s 14s/step - loss: 1.0543 - mse: 1.0543
Epoch 2/30
8/8 [==============================] - 111s 14s/step - loss: 1.1598 - mse: 1.1598
Epoch 3/30
8/8 [==============================] - 111s 14s/step - loss: 0.4384 - mse: 0.4384
Epoch 4/30
8/8 [==============================] - 113s 14s/step - loss: 0.4192 - mse: 0.4192
Epoch 5/30
8/8 [==============================] - 111s 14s/step - loss: 0.4028 - mse: 0.4028
Epoch 6/30
8/8 [==============================] - 112s 14s/step - loss: 0.4173 - mse: 0.4173
Epoch 7/30
8/8 [==============================] - 111s 14s/step - loss: 0.4056 - mse: 0.4056
Epoch 8/30
8/8 [==============================] - 113s 14s/step - loss: 0.4267 - mse: 0.4267
Epoch 9/30
8/8 [==============================] - 111s 14s/step - loss: 0.4009 - mse: 0.4009
Epoch 10/30
8/8 [==============================] - 110s 14s/step - loss: 0.4048 - mse: 0.4048
Epoch 11/30
8/8 [==============================] - 111s 14s/step - lo

In [72]:
model.evaluate([x_test, target_xtest], y_test)

4/4 [==============================] - 12s 3s/step - loss: 0.3989 - mse: 0.3989


[0.3988601565361023, 0.3988601565361023]

In [28]:
y_test.head()

,theta
3,1.00
184,0.00
302,-1.00
239,-0.50
50,0.75


In [73]:
prediction = model.predict([x_test, target_xtest])
# prediction = np.rint(prediction)

In [74]:
print(prediction)

[[-0.03188336]
 [ 0.00914084]
 [-0.09318174]
 [-0.04803247]
 [-0.00203466]
 [-0.09740642]
 [ 0.02748317]
 [-0.04390462]
 [ 0.03010703]
 [-0.08657096]
 [-0.13793817]
 [-0.05444599]
 [ 0.04946668]
 [ 0.00573944]
 [-0.07695134]
 [-0.01470434]
 [-0.10797944]
 [-0.11021017]
 [-0.07943436]
 [-0.05983481]
 [-0.06298362]
 [-0.05643654]
 [-0.06251619]
 [-0.03402146]
 [-0.07464283]
 [-0.00287849]
 [-0.0669593 ]
 [ 0.03735997]
 [ 0.03182318]
 [-0.12525657]
 [ 0.02087166]
 [-0.10347075]
 [ 0.06758225]
 [-0.10929802]
 [ 0.03685772]
 [-0.0024672 ]
 [ 0.07398522]
 [-0.08331723]
 [-0.1026245 ]
 [ 0.01526519]
 [-0.05024613]
 [ 0.07374408]
 [ 0.09768215]
 [-0.10067534]
 [ 0.02812889]
 [-0.04592271]
 [-0.02307035]
 [-0.08727916]
 [ 0.01075698]
 [-0.07288319]
 [ 0.087308  ]
 [-0.01394879]
 [ 0.02379046]
 [ 0.02012995]
 [-0.06649878]
 [-0.02132892]
 [ 0.0299034 ]
 [-0.01199036]
 [ 0.00378139]
 [-0.01128882]
 [-0.12192337]
 [-0.08393504]
 [-0.04558968]
 [ 0.06663311]
 [-0.03718079]
 [ 0.03378954]
 [-0.00791

In [76]:
true_value = prediction+1

In [81]:
true_value = ((true_value*80)/2) + (-25)

In [82]:
print(true_value)

[[13.724667 ]
 [15.365635 ]
 [11.272732 ]
 [13.078701 ]
 [14.918613 ]
 [11.1037445]
 [16.099327 ]
 [13.243816 ]
 [16.20428  ]
 [11.537163 ]
 [ 9.482475 ]
 [12.822159 ]
 [16.978668 ]
 [15.229576 ]
 [11.9219475]
 [14.411827 ]
 [10.680824 ]
 [10.591591 ]
 [11.822628 ]
 [12.606606 ]
 [12.480656 ]
 [12.742538 ]
 [12.4993515]
 [13.639141 ]
 [12.014286 ]
 [14.884861 ]
 [12.321629 ]
 [16.4944   ]
 [16.272926 ]
 [ 9.989738 ]
 [15.834866 ]
 [10.861172 ]
 [17.703289 ]
 [10.628082 ]
 [16.474308 ]
 [14.90131  ]
 [17.959408 ]
 [11.667313 ]
 [10.89502  ]
 [15.610611 ]
 [12.990154 ]
 [17.94976  ]
 [18.907284 ]
 [10.972984 ]
 [16.125153 ]
 [13.163094 ]
 [14.077187 ]
 [11.508835 ]
 [15.430279 ]
 [12.084671 ]
 [18.492321 ]
 [14.442047 ]
 [15.951618 ]
 [15.805199 ]
 [12.34005  ]
 [14.146843 ]
 [16.196136 ]
 [14.520386 ]
 [15.151257 ]
 [14.548447 ]
 [10.123066 ]
 [11.642601 ]
 [13.1764145]
 [17.665325 ]
 [13.512768 ]
 [16.351582 ]
 [14.6833725]
 [11.329369 ]
 [16.393295 ]
 [14.467022 ]
 [10.856709 ]
 [13.1

In [83]:
true_gt = (((y_test + 1)*(55+25))/2)-25

In [84]:
print(true_gt)

     theta
3     55.0
184   15.0
302  -25.0
239   -5.0
50    45.0
..     ...
14    55.0
253   -5.0
193    5.0
54    45.0
177   15.0

[100 rows x 1 columns]


In [42]:
arr_gt = true_gt.to_numpy()

In [43]:
print(arr_gt)

[[ 55.]
 [ 15.]
 [-25.]
 [ -5.]
 [ 45.]
 [-15.]
 [ 15.]
 [ 55.]
 [ 25.]
 [-25.]
 [  5.]
 [-25.]
 [ -5.]
 [  5.]
 [  5.]
 [-15.]
 [ -5.]
 [ 15.]
 [ 15.]
 [ 15.]
 [ 15.]
 [-25.]
 [ 45.]
 [ 35.]
 [ 55.]
 [ 55.]
 [  5.]
 [ 35.]
 [  5.]
 [  5.]
 [ 35.]
 [ 15.]
 [ 15.]
 [-15.]
 [ 45.]
 [ 35.]
 [ 55.]
 [ -5.]
 [-15.]
 [ -5.]
 [ 15.]
 [ 25.]
 [-25.]
 [ -5.]
 [ 45.]
 [ 55.]
 [ 15.]
 [-15.]
 [ 55.]
 [ 25.]
 [ 15.]
 [ 15.]
 [ -5.]
 [ 45.]
 [ 55.]
 [ 35.]
 [ 45.]
 [ 15.]
 [ 25.]
 [-25.]
 [ -5.]
 [ 15.]
 [  5.]
 [  5.]
 [-15.]
 [ 55.]
 [ 45.]
 [-25.]
 [  5.]
 [-15.]
 [-15.]
 [ -5.]
 [-15.]
 [ 55.]
 [ 25.]
 [ -5.]
 [-25.]
 [ 35.]
 [ 25.]
 [  5.]
 [ 35.]
 [ 45.]
 [-25.]
 [-15.]
 [ 35.]
 [ 15.]
 [ 35.]
 [-25.]
 [  5.]
 [-25.]
 [ 15.]
 [ 45.]
 [-15.]
 [ 55.]
 [-25.]
 [ 55.]
 [ -5.]
 [  5.]
 [ 45.]
 [ 15.]]


In [46]:
count = 0
for i in range(len(arr_gt)):
  if arr_gt[i]==15:
    count+=1

print(count)

17


In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/data65_27.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img.shape

In [ ]:
img2 = cv2.imread('/content/target_image_1.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

In [ ]:
predictionimg = model.predict([img,img2])

In [ ]:
predictionimg